# SIAM - Compilar APK en Google Colab

Este notebook compila la aplicacion SIAM para Android.

**Tiempo estimado:** 15-30 min (primera vez)

---

## 1. Configuracion - EDITA AQUI

In [ ]:
# ============================================
# EDITA ESTA LINEA CON TU REPOSITORIO
# ============================================
GITHUB_REPO = "https://github.com/TU_USUARIO/InventarioKivyMVC.git"

# Si tu repo es privado, usa token:
# GITHUB_REPO = "https://TOKEN@github.com/TU_USUARIO/InventarioKivyMVC.git"

# Rama a compilar
BRANCH = "main"

print(f"Repo: {GITHUB_REPO}")
print(f"Rama: {BRANCH}")

## 2. Montar Google Drive (para guardar el APK)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print('Drive montado')

## 3. Clonar repositorio desde GitHub

In [ ]:
import os
import shutil

WORK_DIR = '/content/InventarioKivyMVC'

# Limpiar si existe
if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)

# Limpiar buildozer cache si existe (para evitar errores)
buildozer_cache = '/root/.buildozer'
if os.path.exists(buildozer_cache):
    shutil.rmtree(buildozer_cache)
    print('Cache de buildozer limpiado')

# Clonar repo
!git clone --branch {BRANCH} --depth 1 {GITHUB_REPO} {WORK_DIR}

os.chdir(WORK_DIR)
print(f'\nProyecto clonado en: {WORK_DIR}')
!ls -la

## 4. Instalar dependencias del sistema

In [ ]:
print('Instalando dependencias (~3 minutos)...')

# Dependencias basicas
!apt-get update -qq
!apt-get install -y -qq \
    python3-pip build-essential git python3-dev \
    ffmpeg libsdl2-dev libsdl2-image-dev libsdl2-mixer-dev libsdl2-ttf-dev \
    libportmidi-dev libswscale-dev libavformat-dev libavcodec-dev zlib1g-dev \
    libgstreamer1.0-dev gstreamer1.0-plugins-base gstreamer1.0-plugins-good \
    zip unzip openjdk-17-jdk autoconf libtool pkg-config \
    libncurses5-dev libncursesw5-dev cmake libffi-dev libssl-dev \
    gettext autopoint libltdl-dev libtool-bin \
    lld m4

# Fix para AM_ICONV: copiar macros de gettext a aclocal
!cp /usr/share/aclocal/iconv.m4 /usr/share/aclocal-1.16/ 2>/dev/null || true
!cp /usr/share/gettext/gettext.h /usr/include/ 2>/dev/null || true

# Crear directorio de macros si no existe y copiar
!mkdir -p /usr/local/share/aclocal
!cp /usr/share/aclocal/*.m4 /usr/local/share/aclocal/ 2>/dev/null || true

print('Dependencias instaladas')

## 5. Instalar Buildozer y python-for-android

In [ ]:
!pip install --upgrade pip -q
!pip install --upgrade buildozer cython==0.29.36 virtualenv -q
!pip install --upgrade python-for-android -q
print('Buildozer y p4a instalados')

## 6. Compilar APK

**Primera vez:** 15-30 min (descarga SDK, NDK)

**Siguientes veces:** 5-10 min

In [ ]:
import os
os.chdir(WORK_DIR)

# Configurar variables de entorno para evitar errores de iconv
os.environ['ACLOCAL_PATH'] = '/usr/share/aclocal:/usr/local/share/aclocal'

# Ejecutar buildozer
!buildozer android debug 2>&1 | tee build.log

## 7. Guardar APK en Drive

In [ ]:
from pathlib import Path
import shutil

bin_path = Path(WORK_DIR) / 'bin'
apks = list(bin_path.glob('*.apk')) if bin_path.exists() else []

if apks:
    apk_path = apks[0]
    size_mb = apk_path.stat().st_size / 1024 / 1024
    print(f'APK: {apk_path.name}')
    print(f'Tamano: {size_mb:.1f} MB')
    
    # Guardar en Drive
    output_dir = '/content/drive/MyDrive/APKs'
    os.makedirs(output_dir, exist_ok=True)
    dest = f'{output_dir}/{apk_path.name}'
    shutil.copy2(apk_path, dest)
    print(f'\nGuardado en: {dest}')
else:
    print('ERROR: No se genero el APK')
    print('\nBuscando errores en el log...')
    !grep -i "error" build.log | tail -20

## 8. Descargar APK (opcional)

In [ ]:
if apks:
    from google.colab import files
    files.download(str(apk_path))